#Makes three TFlite models

##Environments

- Google Colab
- Tensorflow 2.3.1
- Edge TPU Compiler 15.0.340273435

##Explanation

- One model is run on cpu, another is quantized and run on cpu, and the other is quantized and run on edgetpu.
- I trained a keras cnn model using the dataset of one day's average temperature provided by Japan Meteorological Agency.
- These models predict the next temperature based on thirty consecutive temperature data.

##Prepare the dataset

- one day's average temperature for 6911 days provided by Japan Meteorological Agency

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/weather/temp.csv")

- input data is a numpy ndarray
  - shape is (6311,30,1)
  - dtype is numpy.float32

In [ ]:
np_temp = np.asarray(data.loc[:len(data),"平均気温"],dtype=np.float32)

In [ ]:
for i in range(len(np_temp)-30):
  if i==0:
    in_temp = np_temp[i:i+30].reshape(1,30)
    out_temp = np_temp[i+30].reshape(1,1)
  else:
    in_temp = np.concatenate([in_temp,np_temp[i:i+30].reshape(1,30)])
    out_temp = np.concatenate([out_temp,np_temp[i+30].reshape(1,1)])

In [ ]:
in_temp = in_temp.reshape(-1,30,1)

##Build a keras cnn model

In [ ]:
inputs = tf.keras.Input(shape=(30,1))
cnn1 = tf.keras.layers.Conv1D(filters=1,kernel_size=10,strides=1,activation='relu',input_shape=(30,1))
cnn2 = tf.keras.layers.Conv1D(filters=1,kernel_size=5,strides=1,activation='relu')
cnn3 = tf.keras.layers.Conv1D(filters=1,kernel_size=3,strides=1,activation='relu')
dense1 = tf.keras.layers.Dense(units=8,activation='relu')
dense2 = tf.keras.layers.Dense(units=1)
x = cnn1(inputs)
x = cnn2(x)
x = cnn3(x)
x = tf.keras.layers.Flatten()(x)
x = dense1(x)
outputs = dense2(x)
model = tf.keras.Model(inputs=inputs,outputs=outputs)
model.compile(optimizer="Adam",loss="mean_squared_error",
                               metrics="binary_accuracy")

In [ ]:
model.summary()

Model: "functional_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 30, 1)]           0         
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 21, 1)             11        
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 17, 1)             6         
_________________________________________________________________
conv1d_33 (Conv1D)           (None, 15, 1)             4         
_________________________________________________________________
flatten_5 (Flatten)          (None, 15)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 8)                 128       
_________________________________________________________________
dense_25 (Dense)             (None, 1)               

##Train the model

- batch size is 50
- epoch is 30

In [ ]:
model.fit(x=in_temp,y=out_temp,batch_size=50,epochs=30)

Epoch 1/30
139/139 [==============================] - 0s 2ms/step - loss: 336.4339 - binary_accuracy: 0.0000e+00
Epoch 2/30
139/139 [==============================] - 0s 2ms/step - loss: 44.7957 - binary_accuracy: 2.8939e-04
Epoch 3/30
139/139 [==============================] - 0s 2ms/step - loss: 12.5332 - binary_accuracy: 2.8939e-04
Epoch 4/30
139/139 [==============================] - 0s 2ms/step - loss: 11.6015 - binary_accuracy: 2.8939e-04
Epoch 5/30
139/139 [==============================] - 0s 2ms/step - loss: 10.7912 - binary_accuracy: 2.8939e-04
Epoch 6/30
139/139 [==============================] - 0s 2ms/step - loss: 10.1794 - binary_accuracy: 2.8939e-04
Epoch 7/30
139/139 [==============================] - 0s 2ms/step - loss: 9.4582 - binary_accuracy: 2.8939e-04
Epoch 8/30
139/139 [==============================] - 0s 2ms/step - loss: 8.8774 - binary_accuracy: 2.8939e-04
Epoch 9/30
139/139 [==============================] - 0s 2ms/step - loss: 8.3673 - binary_accuracy: 2.893

##Convert the keras model to a TFlite model run on cpu

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp0ookxp8i/assets


INFO:tensorflow:Assets written to: /tmp/tmp0ookxp8i/assets


- save the TFlite model

In [ ]:
open("cnn_weather_lite.tflite","wb").write(tf_lite_model)

5184

##Quantize the keras model

- determine the input shape and save the model as tf model (can't quantize without the constant input shape)

In [ ]:
opt = tf.function(lambda x:model(x))
BACTH_SIZE = 1
STEPS = 30
INPUT_SIZE = 1
concrete_func = opt.get_concrete_function(tf.TensorSpec([BACTH_SIZE,STEPS,INPUT_SIZE],
                                                        model.inputs[0].dtype,name="inputs")
)
model.save('/content/weather',save_format="tf",signatures=concrete_func)

INFO:tensorflow:Assets written to: /content/weather/assets


INFO:tensorflow:Assets written to: /content/weather/assets


- quantize the tf model(details: https://www.tensorflow.org/lite/performance/post_training_integer_quant)

In [ ]:
conv_data = in_temp[0]
conv_data = conv_data.reshape(1,30,1)

In [ ]:
def representative_dataset_gen():
  for i in range(len(conv_data)):
    yield [conv_data[i]]

In [ ]:
converter_edgetpu = tf.lite.TFLiteConverter.from_saved_model("/content/weather")
converter_edgetpu.optimizations = [tf.lite.Optimize.DEFAULT]
converter_edgetpu.representative_dataset = representative_dataset_gen
converter_edgetpu.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_edgetpu.inference_input_type = tf.uint8
converter_edgetpu.inference_output_type = tf.uint8
converter_edgetpu.experimental_new_converter = True
tflite = converter_edgetpu.convert()

- save the quantized TFlite model

In [ ]:
open("cnn_weather_lite_quantized.tflite","wb").write(tflite)

8560

- install edgetpu compiler

In [ ]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  12557      0 --:--:-- --:--:-- --:--:-- 12557
OK


In [ ]:
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

deb https://packages.cloud.google.com/apt coral-edgetpu-stable main


In [ ]:
!sudo apt -y update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [6,332 B]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 https://packages.cloud.google.com/apt coral-edgetpu-stable/main amd64 Packages [2,172 B]
Hit:14 http://ppa.launchpad.net/graphics-drivers/pp

In [ ]:
!sudo apt-get install edgetpu-compiler

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  edgetpu-compiler
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 7,865 kB of archives.
After this operation, 31.2 MB of additional disk space will be used.
Get:1 https://packages.cloud.google.com/apt coral-edgetpu-stable/main amd64 edgetpu-compiler amd64 15.0 [7,865 kB]
Fetched 7,865 kB in 1s (6,350 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package edgetpu-compiler.
(Reading datab

- compile the quantized TFlite model for edgetpu

In [ ]:
!edgetpu_compiler /content/cnn_weather_lite_quantized.tflite

Edge TPU Compiler version 15.0.340273435

Model compiled successfully in 56 ms.

Input model: /content/cnn_weather_lite_quantized.tflite
Input size: 8.36KiB
Output model: cnn_weather_lite_quantized_edgetpu.tflite
Output size: 48.50KiB
On-chip memory used for caching model parameters: 6.75KiB
On-chip memory remaining for caching model parameters: 7.85MiB
Off-chip memory used for streaming uncached model parameters: 192.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 17
Operation log: cnn_weather_lite_quantized_edgetpu.log
See the operation log file for individual operation details.
